In [17]:
import sys
sys.path.append("../../")

import os
from PIL import Image
from pathlib import Path
import requests
import json

from utils_cv.common.image import im2base64, ims2strlist
from utils_cv.common.data import data_path
from utils_cv.detection.model import DetectionLearner, get_pretrained_fasterrcnn
from utils_cv.detection.dataset import DetectionDataset
from utils_cv.detection.plot import (
    display_bboxes,
    PlotSettings,
)

In [18]:
%load_ext blackcellmagic

# Ensure edits to libraries are loaded and plotting is shown in the notebook.
%reload_ext autoreload
%autoreload 2
%matplotlib inline

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [ ]:
# detector = DetectionLearner.from_saved_model(name="my_drink_detector", path=data_path()/"odFridgeObjects"/"models")

# # new_im_path = Path("/home/jiata/code/computervision/data/cvbp_milk_bottle.jpg")
# new_im_path = Path(data_path()/"odFridgeObjects"/"images"/"1.jpg")

# new_im = Image.open(new_im_path)
# detections = detector.predict(new_im, threshold=0.5)

# plot_settings = PlotSettings(rect_color=(0, 255, 0))
# display_bboxes(
#     detections,
#     new_im_path,
#     plot_settings=plot_settings,
# )
# detections

---

In [ ]:
# # Extract test images paths
# im_url_root = "https://cvbp.blob.core.windows.net/public/images/"
# im_filenames = ["cvbp_milk_bottle.jpg", "cvbp_water_bottle.jpg"]

# for im_filename in im_filenames:
#     # Retrieve test images from our storage blob
#     r = requests.get(os.path.join(im_url_root, im_filename))

#     # Copy test images to local data/ folder
#     with open(os.path.join(data_path(), im_filename), 'wb') as f:
#         f.write(r.content)

# # Extract local path to test images
# local_im_paths = [os.path.join(data_path(), im_filename) for im_filename in im_filenames]

# # Convert images to json object
# im_string_list = ims2strlist(local_im_paths)
# service_input = json.dumps({"data": im_string_list})

In [ ]:
# from base64 import b64decode
# from io import BytesIO

# def test_img(raw_data):

#     # Expects raw_data to be a list within a json file
#     results = []    
#     for im_string in json.loads(raw_data)['data']:
#         im_bytes = b64decode(im_string)
#         im_dets = []
#         try:
#             new_im = Image.open(BytesIO(im_bytes))
#             detections = detector.predict(new_im, threshold=0.5)
#             for d in detections:
#                 im_dets.append({
#                     "top": str(d.top), 
#                     "left": str(d.left),
#                     "bottom": str(d.bottom), 
#                     "right": str(d.right),
#                     "label_name": d.label_name,
#                     "label_idx": str(d.label_idx),
#                     "score": str(d.score)
#                 })
#         except Exception as e:
#             im_dets.append({"error": str(e)})
#         results.append(im_dets)
#     return results

In [ ]:
# from utils_cv.detection.bbox import DetectionBbox

# ret = test_img(service_input)
# for im in ret:
#     dets = []
#     for bbox in im:
#         det = DetectionBbox.from_array(
#             [bbox['left'], bbox['top'], bbox['right'], bbox['bottom']],
#             label_name=bbox['label_name'],
#             label_idx=bbox['label_idx'],
#             score=bbox['score']
#         )
#         dets.append(det)
#     print(dets)
        

---

In [ ]:
# Azure resources
subscription_id = "989b90f7-da4f-41f9-84c9-44848802052d"
resource_group = "jiata-cv-aml"  
workspace_name = "jiata-cv-aml"  
workspace_region = "southcentralus" #Possible values eastus, eastus2, etc.

# Choose a size for our cluster and the maximum number of nodes
VM_SIZE = "Standard_NC6s_v3" #"Standard_NC6s_v3", "Standard_NC6S_V3", "Standard_D3_v2"
NODES = 3
KUBERNETES_CLUSTER_NAME = 'jiata-aks-gpu0'

In [ ]:
from utils_cv.common.azureml import get_or_create_workspace

ws = get_or_create_workspace(
    subscription_id, resource_group, workspace_name, workspace_region
)

# Print the workspace attributes
print(
    "Workspace name: " + ws.name,
    "Workspace region: " + ws.location,
    "Subscription id: " + ws.subscription_id,
    "Resource group: " + ws.resource_group,
    sep="\n",
)

Register the model

In [ ]:
from azureml.core.model import Model

model_state_dict = Model.register(
    model_path=os.path.join(
        data_path(), "odFridgeObjects", "models", "my_drink_detector", "model.pt"
    ),
    model_name="jiata-dd-model-state-dict-0",
    tags={"Model": "object_detection"},
    description="state dict for drink detector",
    workspace=ws,
)

model_meta = Model.register(
    model_path=os.path.join(
        data_path(), "odFridgeObjects", "models", "my_drink_detector", "meta.json"
    ),
    model_name="jiata-dd-model-meta-0",
    tags={"Model": "object_detection"},
    description="meta data for drink detector",
    workspace=ws,
)

Create scoring script.

In [ ]:
scoring_script = "score.py"

In [ ]:
%%writefile $scoring_script
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license.

import os
import json
import shutil

from pathlib import Path
from base64 import b64decode
from io import BytesIO
from PIL import Image

from azureml.core.model import Model
from utils_cv.detection.model import DetectionLearner, get_pretrained_fasterrcnn
from utils_cv.detection.dataset import DetectionDataset


def init():
    global detector
    # retrieve the path to the model file using the model name
    model_state_dict_path = Model.get_model_path(model_name='jiata-dd-model-state-dict-0')
    model_meta_data_path = Model.get_model_path(model_name='jiata-dd-model-meta-0')

    # put model.pt and meta.json into the same folder
    model_dir = "new_detector"
    os.mkdir(model_dir)
    shutil.copyfile(model_state_dict_path, os.path.join(os.getcwd(), model_dir, "model.pt"))
    shutil.copyfile(model_meta_data_path, os.path.join(os.getcwd(), model_dir, "meta.json"))
    
    # load up the model
    detector = DetectionLearner.from_saved_model(name=model_dir, path=Path(os.getcwd()))


def run(raw_data):
    # Expects raw_data to be a list within a json file
    results = []    
    for im_string in json.loads(raw_data)['data']:
        im_bytes = b64decode(im_string)
        im_dets = []
        try:
            new_im = Image.open(BytesIO(im_bytes))
            detections = detector.predict(new_im, threshold=0.5)
            for d in detections:
                im_dets.append({
                    "top": str(d.top), 
                    "left": str(d.left),
                    "bottom": str(d.bottom), 
                    "right": str(d.right),
                    "label_name": d.label_name,
                    "label_idx": str(d.label_idx),
                    "score": str(d.score)
                })
        except Exception as e:
            im_dets.append({"error": str(e)})
        results.append(im_dets)
    return results

Scoring script:

In [ ]:
%%writefile "myenv.yml"
name: cv-remote
channels:
- defaults
- conda-forge
dependencies:
- python==3.6.8
- pytorch>=1.2.0
- torchvision>=0.3.0
- cython>=0.29.1
- git
- pip:
  - git+https://github.com/microsoft/ComputerVision.git@staging#egg=utils_cv
  - git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

In [ ]:
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException

# Configure the Docker image
try:
    image_config = ContainerImage.image_configuration(
        execution_script="score.py",
        runtime="python",
        conda_file="myenv.yml",
        description="Image with FasterRCNN model",
        tags={
            "scenario": "detection",
            "architecture": "FasterRCNN",
            "type": "Pretrained",
        },
        base_image="mcr.microsoft.com/azureml/onnxruntime:latest-cuda",
    )
except WebserviceException:
    raise FileNotFoundError(
        "The files *score.py* and/or *myenv.yaml* could not be found - Please run the cells above again"
    )

In [ ]:
# Create the Docker image
try:
    docker_image = ContainerImage.create(
        name="object-detection-fridge-objects",
        models=[model_state_dict, model_meta],
        image_config=image_config,
        workspace=ws,
    )
    # The image name should not contain more than 32 characters, and should not contain any spaces, dots or underscores
except WebserviceException:
    raise FileNotFoundError(
        "The files *score.py* and/or *myenv.yaml* could not be found - Please run the cells above again"
    )

In [ ]:
%%time
docker_image.wait_for_creation(show_output=True)  # This can take up to 12 min

In [ ]:
print(ws.images["object-detection-fridge-objects"].image_build_log_uri)

---

First we create our Kubernetes cluster

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

if KUBERNETES_CLUSTER_NAME not in ws.compute_targets:
    # Configure the cluster using the default configuration (i.e. with 3 virtual machines)
    prov_config = AksCompute.provisioning_configuration(
        vm_size=VM_SIZE, agent_count=NODES
    )

    # Create the cluster
    aks_target = ComputeTarget.create(
        workspace=ws,
        name=KUBERNETES_CLUSTER_NAME,
        provisioning_configuration=prov_config,
    )
    aks_target.wait_for_completion(show_output=True)
    print(f"We created the {aks_target.name} AKS compute target")
else:
    # Retrieve the already existing cluster
    aks_target = ws.compute_targets[KUBERNETES_CLUSTER_NAME]
    print(f"We retrieved the {aks_target.name} AKS compute target")

Creating..............................................................................

In [ ]:
print("Docker images:")
for docker_im in ws.images:    
    print(f" --> Name: {ws.images[docker_im].name}\n \
    --> ID: {ws.images[docker_im].id}\n \
    --> Tags: {ws.images[docker_im].tags}\n \
    --> Creation time: {ws.images[docker_im].created_time}\n"
         )

In [ ]:
docker_image

In [ ]:
from azureml.core.webservice import AksWebservice, LocalWebservice
aks_config = AksWebservice.deploy_configuration(enable_app_insights=True)

In [ ]:
aks_service_name = "aks-gpu-od-web-svc0"
aks_service = Webservice.deploy_from_image(
    workspace=ws,
    name=aks_service_name,
    image=docker_image,
    deployment_config=aks_config,
    deployment_target=aks_target,
)
aks_service.wait_for_deployment(show_output=True)
print(f"The web service is {aks_service.state}")

In [ ]:
print(aks_service.get_logs())

local testing

testing

In [ ]:
# Extract test images paths
im_url_root = "https://cvbp.blob.core.windows.net/public/images/"
im_filenames = ["cvbp_milk_bottle.jpg", "cvbp_water_bottle.jpg"]

for im_filename in im_filenames:
    # Retrieve test images from our storage blob
    r = requests.get(os.path.join(im_url_root, im_filename))

    # Copy test images to local data/ folder
    with open(os.path.join(data_path(), im_filename), 'wb') as f:
        f.write(r.content)

# Extract local path to test images
local_im_paths = [os.path.join(data_path(), im_filename) for im_filename in im_filenames]

# Convert images to json object
im_string_list = ims2strlist(local_im_paths)
service_input = json.dumps({"data": im_string_list})

In [ ]:
result = ws.webservices['aks-cpu-od-web-svc6'].run(service_input)

In [ ]:
result